# Tier 1: Put-Call Parity

**Learning Objectives:**
- Understand the put-call parity relationship
- Use it to verify BS implementations
- Identify common implementation bugs

**Key Formula [T1]:**
$$C - P = S e^{-qT} - K e^{-rT}$$

**Duration:** ~15 minutes

In [ ]:
import numpy as np
from scipy.stats import norm

from annuity_pricing.options.pricing.black_scholes import (
    black_scholes_call,
    black_scholes_put,
)

## 1. The Relationship

Put-call parity is NOT a model — it's an **arbitrage relationship**.

If violated, you can make risk-free profit:
- If C - P > S - Ke^(-rT): Sell call, buy put, buy stock
- If C - P < S - Ke^(-rT): Buy call, sell put, sell stock

In [ ]:
# Hull Example 15.6 parameters
S = 42.0
K = 40.0
r = 0.10
q = 0.0
sigma = 0.20
T = 0.5

# Prices
call = black_scholes_call(S, K, r, q, sigma, T)
put = black_scholes_put(S, K, r, q, sigma, T)

# Put-call parity
lhs = call - put
rhs = S * np.exp(-q * T) - K * np.exp(-r * T)

print(f"Hull Example 15.6:")
print(f"  Call = ${call:.4f} (expected ~$4.76)")
print(f"  Put  = ${put:.4f} (expected ~$0.81)")
print()
print(f"Put-Call Parity:")
print(f"  C - P = ${lhs:.6f}")
print(f"  S*exp(-qT) - K*exp(-rT) = ${rhs:.6f}")
print(f"  Difference = {abs(lhs - rhs):.2e}")
print(f"  Status: {'✓ VALID' if abs(lhs - rhs) < 1e-10 else '✗ VIOLATED'}")

## 2. Common Bugs That Violate Parity

| Bug | Symptom | Fix |
|-----|---------|-----|
| Wrong d1 sign | Large parity violation | Check +0.5σ² not -0.5σ² |
| Missing q term | Parity off by exp(-qT) | Include dividend yield |
| Missing sqrt(T) | Extreme prices | Use σ√T not σT |

In [ ]:
def bs_call_buggy(S, K, r, q, sigma, T):
    """BS call with BUG: wrong sign in d1."""
    # BUG: Using -0.5*sigma^2 instead of +0.5*sigma^2
    d1 = (np.log(S/K) + (r - q - 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def bs_put_buggy(S, K, r, q, sigma, T):
    """BS put with same BUG."""
    d1 = (np.log(S/K) + (r - q - 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1)

# Test buggy implementation
call_bug = bs_call_buggy(S, K, r, q, sigma, T)
put_bug = bs_put_buggy(S, K, r, q, sigma, T)
lhs_bug = call_bug - put_bug

print("Buggy Implementation (wrong d1 sign):")
print(f"  Call = ${call_bug:.4f} (correct: ${call:.4f})")
print(f"  Put  = ${put_bug:.4f} (correct: ${put:.4f})")
print(f"  C - P = ${lhs_bug:.6f}")
print(f"  Expected = ${rhs:.6f}")
print(f"  Parity violation = ${abs(lhs_bug - rhs):.6f}")
print(f"  Status: {'✗ BUG DETECTED' if abs(lhs_bug - rhs) > 0.01 else '✓ OK'}")

## 3. Use Parity as a Test

**Always** verify put-call parity in your test suite:

In [ ]:
def test_put_call_parity(S, K, r, q, sigma, T, tol=1e-10):
    """Test that put-call parity holds."""
    call = black_scholes_call(S, K, r, q, sigma, T)
    put = black_scholes_put(S, K, r, q, sigma, T)
    
    lhs = call - put
    rhs = S * np.exp(-q * T) - K * np.exp(-r * T)
    
    return abs(lhs - rhs) < tol

# Test with various parameters
test_cases = [
    (100, 100, 0.05, 0.02, 0.20, 1.0),  # ATM
    (100, 80, 0.05, 0.02, 0.20, 1.0),   # ITM call
    (100, 120, 0.05, 0.02, 0.20, 1.0),  # OTM call
    (100, 100, 0.05, 0.02, 0.40, 2.0),  # High vol, long term
]

print("Put-Call Parity Tests:")
for S, K, r, q, sigma, T in test_cases:
    passes = test_put_call_parity(S, K, r, q, sigma, T)
    print(f"  S={S}, K={K}, σ={sigma:.0%}, T={T}: {'✓' if passes else '✗'}")

## 4. Key Takeaways

1. **Put-call parity is fundamental** — violations indicate bugs
2. **Test early and often** — add parity tests to your suite
3. **Tolerance ~1e-10** for numerical precision

## Next

Continue to `03_arbitrage_bounds.ipynb` for more price constraints.